In [ ]:
# es geht darum die verschiedenen Sentece Transformer anzuwenden, um heruaszufinden welche besser funktioniert
# äEs soll darum gehen die 2 Größten und best performanten miteinander zu vergleichen und eine ST die auf deutsche Daten traniert wurde

In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import hnswlib
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np
import random

/var/folders/ts/4_j3m5qs2wvgc9jjm_yb5p1h0000gn/T/ipykernel_30432/433430082.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/mohammadrezaghassemzadeh/Desktop/p-matching/product-matching/bachelorarbeit/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_amazon = r'daten/amazon_datenbereinigt.xlsx'
df_amazon = pd.read_excel(data_amazon)


In [4]:
data_otto = r'daten/otto_datenbereinigt.xlsx'
df_otto = pd.read_excel(data_otto)


In [5]:
# separator = " | "  # Trennzeichen
# Amazon data
df_amazon_sentence = df_amazon.apply(lambda row: ''.join(row[col] for col in ['NAME', 'DESCRIPTION'] 
                                                          if not pd.isna(row[col])), axis=1).tolist()

In [6]:
# Otto data
df_otto_sentence = df_otto.apply(lambda row: ''.join(row[col] for col in ['NAME','DESCRIPTION'] 
                                                      if not pd.isna(row[col])), axis=1).tolist()

In [8]:
model_zweitbeste  = SentenceTransformer('all-MiniLM-L12-v2') 

In [8]:
df_amazon_embeddings = model_zweitbeste.encode(df_amazon_sentence, batch_size = 32)

In [11]:
df_otto_embeddings = model_zweitbeste.encode(df_otto_sentence, batch_size = 32)

In [13]:
# Modell Laden https://aclanthology.org/2022.bucc-1.9.pdf quelle !!!
# zwei beste model im vergleich 
dim = df_otto_embeddings.shape[1]  # Vektor-Dimension
index = hnswlib.Index(space='l2', dim=dim)# l2 für Euklid

index.init_index(max_elements=len(df_otto), ef_construction=200, M=16)

index.add_items(df_otto_embeddings)

index.save_index('_2bestemodell_hnswlib_index_otto.bin')
index.set_ef(100) 

similar_items = []
for i, embedding in enumerate(df_amazon_embeddings):
    # Abfragen der fünf nächsten Nachbarn (das eigentliche Produkt eingeschlossen 1+)
    labels, distances = index.knn_query(embedding, k=5)
    # Speichern der ähnlichsten Produkte
    similar_products = [df_otto_sentence[j] for j in labels[0]]
    similar_items.append(similar_products)

df = pd.DataFrame(similar_items, columns=['amazon_product',
                                          'otto_product_1', 
                                          'otto_product_2',
                                          'otto_product_3',
                                          'otto_product_4'
                                          ])
df.to_excel("daten_sentence_tranformer_im_vergleich/all_distil_robertav1_2beste_amazon_otto_daten.xlsx")
df.head(10)

NameError: name 'df_otto_embeddings' is not defined

In [16]:
def negative_examples(df, num_negatives_per_positive=1):
    """
    :return: DataFrame mit den negativen Beispielen.
    """
    negative_examples = []
    product_names = df['amazon_product_name'].tolist() + df['otto_variation_name'].tolist()
    
    for _, row in df.iterrows(): # der index selbst wird ignoriert 
        for _ in range(num_negatives_per_positive):
            # zuffälig zwei Produkte nehmen, die nicht paare sind 
            while True:
                product1 = random.choice(product_names)
                product2 = random.choice(product_names)
                
                # hierit wird sichergestellt, dass product1 und product2 nicht übereinstimmen und nicht in den positiven Beispielen sind
                if product1 != product2 and not ((df['amazon_product_name'] == product1) & (df['otto_variation_name'] == product2)).any():
                    break
            
            negative_examples.append({'amazon_product_name': product1, 'otto_variation_name': product2})
    
    return pd.DataFrame(negative_examples)

# schwere Negative bsp bauen!!!


In [17]:
import pandas as pd
import random

def harder_negative_examples(df, num_negatives_per_positive=1):
    """
    hierbei sind die Produkte ähnlich, aebr nicht identisch 
    :return: DataFrame mit den negativen Beispielen.
    """
    negative_examples = []
    categories = df['kategorie_bezeichnung'].unique()

    for _, row in df.iterrows():
        for _ in range(num_negatives_per_positive):

            current_category = row['kategorie_bezeichnung']
            
            # Produkte innerhalb der gleichen Kategorie, aber nicht das gleiche Produkt
            similar_products = df[(df['kategorie_bezeichnung'] == current_category) & 
                                  (df['amazon_product_name'] != row['amazon_product_name']) &
                                  (df['otto_variation_name'] != row['otto_variation_name'])]
            
            # Wenn es ähnliche Produkte gibt, wird eines zufällig als negatives Beispiel gewählt
            if not similar_products.empty:
                negative_example = similar_products.sample(1)
                product1 = row['amazon_product_name']
                product2 = negative_example['otto_variation_name'].values[0]
                
                negative_examples.append({'amazon_product_name': product1, 'otto_variation_name': product2})

    return pd.DataFrame(negative_examples)

In [18]:
df= pd.read_pickle(r'daten/otto_amazon_variation_pairs.pkl', compression='gzip')

In [ ]:
positive_embeddings_1 = model_zweitbeste.encode(df['amazon_product_name'].tolist())
positive_embeddings_2 = model_zweitbeste.encode(df['otto_variation_name'].tolist())
# Absolute Differenz als Features berechnen 
positive_features = np.abs(positive_embeddings_1 - positive_embeddings_2)

In [33]:
negative_df = harder_negative_examples(df)
negative_embeddings_1 = model_zweitbeste.encode(negative_df['amazon_product_name'].tolist()) #eventuell nochmal mit batchsize machen
negative_embeddings_2 = model_zweitbeste.encode(negative_df['otto_variation_name'].tolist())


In [34]:
negative_features = np.abs(negative_embeddings_1 - negative_embeddings_2)
# Features und Labels zusammenführen Positive Features
positive_labels = np.ones(len(positive_features))
negative_labels = np.zeros(len(negative_features))
# Features und Labels zusammenführen
X = np.concatenate([positive_features, negative_features], axis=0)
y = np.concatenate([positive_labels, negative_labels], axis=0)

# Trainings- und Testsets aufteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)

print(y_train.shape)

""" # SVM trainieren
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Vorhersagen und Evaluation 
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred)) """
# Svm aber andere Sentence Transformer Modell 

(101269,)
              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91     12556
         1.0       0.92      0.90      0.91     12762

    accuracy                           0.91     25318
   macro avg       0.91      0.91      0.91     25318
weighted avg       0.91      0.91      0.91     25318



In [37]:
from sklearn import tree

clf_tree = tree.DecisionTreeClassifier()
clf_tree = clf_tree.fit(X_train, y_train)
y_pred = clf_tree.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.75      0.74      0.75     12556
         1.0       0.75      0.76      0.75     12762

    accuracy                           0.75     25318
   macro avg       0.75      0.75      0.75     25318
weighted avg       0.75      0.75      0.75     25318



In [43]:
from sklearn.linear_model import SGDClassifier

clf_GDC = SGDClassifier(loss="modified_huber", max_iter=3000, tol=1e-3)
clf_GDC.fit(X_train, y_train)
y_pred = clf_GDC.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.88      0.94      0.91     12556
         1.0       0.94      0.88      0.91     12762

    accuracy                           0.91     25318
   macro avg       0.91      0.91      0.91     25318
weighted avg       0.91      0.91      0.91     25318



In [45]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier()
clf_rf  = clf_rf.fit(X_train, y_train)
y_pred = clf_rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.84      0.96      0.90     12556
         1.0       0.96      0.82      0.88     12762

    accuracy                           0.89     25318
   macro avg       0.90      0.89      0.89     25318
weighted avg       0.90      0.89      0.89     25318



In [35]:
# Modell speichern
from joblib import dump
dump(clf, 'modell/zweite_svm_modell_hard_Negativ_all-MiniLM-L12-v2.joblib')


['modell/zweite_svm_modell_hard_Negativ_all-MiniLM-L12-v2.joblib']

In [ ]:
'''--------------------------------------------------------------------------------------------------------------------------------------'''

In [4]:
df_amazon = pd.read_excel("daten/amazon_datenbereinigt.xlsx")

df_otto = pd.read_excel("daten/otto_datenbereinigt.xlsx")

In [5]:
amazon_text = df_amazon['NAME'].tolist()
otto_text = df_otto['NAME'].tolist()

In [20]:
amazon_embeddings = model_zweitbeste.encode(amazon_text, batch_size=32)

In [21]:
otto_embeddings = model_zweitbeste.encode(otto_text, batch_size=32)

In [24]:
from joblib import load

# Laden des trainierten Modells
clf = load('modell/zweite_svm_modell_hard_Negativ_all-MiniLM-L12-v2.joblib')


In [25]:
# Optimiertes Modell
ergebnisse = []  # Ergebnisse sollten als Liste initialisiert werden
for index, amazon_emb in enumerate(amazon_embeddings):
    distances = []
    # Ändern Sie diesen Teil, um über alle Otto-Embeddings zu iterieren
    for otto_emb in otto_embeddings:  # Entfernen Sie 'index, i' und iterieren Sie direkt über otto_embeddings
        
        # Feature für das Paar berechnen
        feature = np.abs(amazon_emb - otto_emb).reshape(1, -1)
        
        # Distanz zur Entscheidungsgrenze berechnen und normalisieren
        distance = clf.decision_function(feature)
        normalized_distance = sigmoid(distance)
        distances.append(normalized_distance)
    
    # Ähnlichste Otto-Produkt bestimmen
    most_similar_index = np.argmax(distances)
    # Speichern Sie das Ergebnis. Beachten Sie, dass wir nicht mehr df.at[index, f'otto_product_{most_similar_index+1}'] verwenden,
    # da dies davon ausging, dass es nur vier otto_products pro Amazon-Produkt gibt
    ergebnisse.append({
        'amazon_product_index': index,  # Speichern Sie den Index des Amazon-Produkts
        'most_similar_otto_product_index': most_similar_index,  # Speichern Sie den Index des ähnlichsten Otto-Produkts
        'normalized_distance_score': distances[most_similar_index]  # Extrahieren des Skalarwerts
    })


ValueError: X has 384 features, but SVC is expecting 768 features as input.

In [ ]:
# SVM trainieren
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Vorhersagen und Evaluation 
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [25]:
model_finetuent_german = SentenceTransformer('all-distilroberta-v1')

In [26]:
# Weitere Modell um Sentence transformer modelle miteinander anhand der Ergebnisse zu vergleichen: auf deutsche Daten fine tuent 
df_amazon_embeddings = model_finetuent_german.encode(df_amazon_sentence, batch_size = 32)

In [29]:
df_otto_embeddings = model_finetuent_german.encode(df_otto_sentence, batch_size = 32)


KeyboardInterrupt: 

In [ ]:

dim = df_otto_embeddings.shape[1] 
index = hnswlib.Index(space='l2', dim=dim)# l2 für Euklid

index.init_index(max_elements=len(df_otto), ef_construction=200, M=16)

index.add_items(df_otto_embeddings)

index.save_index('_deutsche_bestemodell_hnswlib_index_otto.bin')
index.set_ef(100) 

similar_items = []
for i, embedding in enumerate(df_amazon_embeddings):
    # Abfragen der fünf nächsten Nachbarn (das eigentliche Produkt eingeschlossen 1+)
    labels, distances = index.knn_query(embedding, k=5)
    # Speichern der ähnlichsten Produkte
    similar_products = [df_otto_sentences[j] for j in labels[0]]
    similar_items.append(similar_products)

df = pd.DataFrame(similar_items, columns=['amazon_product',
                                          'otto_product_1', 
                                          'otto_product_2',
                                          'otto_product_3',
                                          'otto_product_4'
                                          ])
df.to_excel("daten_sentence_tranformer_im_vergleich/T-Systems-onsite_deutschmodell_amazon_otto_daten.xlsx")
df.head(10)

In [ ]:
# Präziste Model! Quelle : https://www.sbert.net/docs/pretrained_models.html
model_beste = sentence_transformers('sentence-transformers/all-mpnet-base-v2')